# Huggy Training Center

## Configuring the virtual machine

In [ ]:
import os
import uuid

In [ ]:
%%capture
# Installing ml-agents
!cd
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents
!cd ..

In [ ]:
%%capture
# Downloading the Huggy environment
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux
!gdown --id 1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF -O './trained-envs-executables/linux/Huggy.zip'
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip
!chmod -R 755 ./trained-envs-executables/linux/Huggy

## Configuration of the filepaths

We created this notebook such that we can connect it to a drive file system in order to load configuration files and save the training output. If one runs this notebook as a demo, we recommend leaving `USE_DRIVE_FILESYSTEM = False. This will let the training to run locally on this virutal machine. The training will however be lost when the machine is shut down.

In [ ]:
USE_DRIVE_FILESYSTEM = False
WORKING_PATH = None

In [ ]:
if USE_DRIVE_FILESYSTEM:
  from google.colab import drive
  drive.mount('/content/drive/MyDrive/deep_learning')

  if os.path.exists("/content/drive/MyDrive/deep_learning"):
    DRIVE_PATH = "/content/drive/MyDrive/deep_learning"
  elif os.path.exists("/content/drive/MyDrive/shared/deep_learning"):
    DRIVE_PATH = "/content/drive/MyDrive/shared/deep_learning"
  else:
    DRIVE_PATH = None
    raise Warning("Drive filepath not found")
    print("Drive filepath not found, data will be saved on the virtual machine")
  WORKING_PATH = DRIVE_PATH

# If no result path previously indicated, defaulting to saving data on the virutal machine
if WORKING_PATH is None:
  os.system("mkdir working_directory")
  os.system("mkdir working_directory/training_output")
  os.system("mkdir working_directory/training_output/OK")
  os.system("mkdir working_directory/training_output/KO")
  os.system("mkdir working_directory/configuration_files")
  os.system("mkdir working_directory/configuration_files/training")
  os.system("mkdir working_directory/configuration_files/to_train")
  os.system("mkdir working_directory/configuration_files/done_training")

  WORKING_PATH = "working_directory"


print(f"Working path: {WORKING_PATH}")

## Loading a baseline PPO training configuration file from the internet into our working directory

This training configuration file has a standard configuration but with only 20000 steps for demonstration purposes (training should take less than a minute).

One can skip this step and load their own configuration file to `[WORKING_PATH]/configuration_files/to_train` instead.

In [ ]:
%%capture
os.system(f"gdown --id 1gOHrXTM_fVQKUrKAmlCCvtW2g07pUHB_ -O '{WORKING_PATH}/configuration_files/to_train/ppo_baseline_short.yaml'")

## Seting up training routine

In [ ]:
def train_huggy(
    configuration_filename,
    configuration_filepath=f"{WORKING_PATH}/configuration_files",
    environment="./trained-envs-executables/linux/Huggy/Huggy",
    output_folder=f"{WORKING_PATH}/training_output",
    move_configuration=True,
    ):

  # Generating a unique id for the training
  training_id = f"{configuration_filename[:-5]}_{uuid.uuid4().hex}"

  print(f"Running training: {training_id}")

  # Starting up the training
  if move_configuration:
    os.system(f"cp {configuration_filepath}/to_train/{configuration_filename} {configuration_filepath}/training/")

  run_ml_agents_command = f'mlagents-learn "{configuration_filepath}/to_train/{configuration_filename}" --env="{environment}" --run-id="{training_id}" --results-dir "{output_folder}/OK" --no-graphics'

  print("mlagent-learn command", run_ml_agents_command)

  # Start training
  is_training_OK = False
  try:
     train_command_output = os.system(run_ml_agents_command)
     is_training_OK = True
  except Exception as e:
    print(f"Error while training\nid: {training_id}\nerror:\n{e}")

  # Save the output data to the relevant folder:
  if is_training_OK:

    if move_configuration:
      os.system(f"mv {configuration_filepath}/training/{configuration_filename} {configuration_filepath}/done_training/")
      os.system(f"rm {configuration_filepath}/to_train/{configuration_filename} {configuration_filepath}/training/")


    os.system(f"echo '{train_command_output}' >> '{output_folder}/OK/{training_id}/training_log.txt'")
    print(f"Successfuly ran training. Saved output to {output_folder}/{training_id}")
  else:
    os.system(f"cp -r './results/{training_id}' '{output_folder}/KO'")
    if move_configuration:
      os.system(f"mv {configuration_filepath}/training/{configuration_filename} {configuration_filepath}/to_train/")
      os.system(f"rm {configuration_filepath}/to_train/{configuration_filename} {configuration_filepath}/training/")

  return training_id


## Loading the configuration file and starting the training

In [ ]:
training_configurations = os.listdir(f"{WORKING_PATH}/configuration_files/to_train")

if len(training_configurations) > 0:
  current_configuration = training_configurations.pop()
  print(current_configuration)
else:
  print("no configuration file found")

In [ ]:
training_id = train_huggy(current_configuration, move_configuration=True)

## Looking at the training output:

In [ ]:
!ls working_directory/training_output/OK


In [ ]:
os.listdir(f"{WORKING_PATH}/training_output/OK/{training_id}")

This shows the output directory. It contains logs, the configuration file used, as well as the Huggy.onnx model file. Our training has been successful. It is now possible to load the .onnx model into the Huggy Unity environment to test out our model. Alternatively, we can look at the logs to see how the training went. This can be done using tensorboard:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir working_directory/training_output/OK/